In [1]:
import sys
if '../src' not in sys.path:
    sys.path.insert(0, '../src')
    sys.path.insert(0, '../src/lib')
    sys.path.insert(0, '../../.conda/envs/mktrack/lib/python3.8/site-packages')

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision
from torchmetrics.detection import MeanAveragePrecision

import datasets.dataset as datasets
from models.model import create_model, load_model
from models.decode import mot_decode, map2orig

from opts import opts

In [2]:
models = ['macaquepose', 'macaquecp', 'macaquecpw']
seeds = ['1','2','3']
valsets = ['MacaquePose', 'MacaqueCopyPaste', 'MacaqueCopyPasteWild']

In [ ]:
df = pd.DataFrame(columns = ['model', 'valset', 'seed', 'AP', 'AP50', 'AP75'])


for mdl in models:
    for valset in valsets:
        for seed in seeds:
            epoch = 200
            print(mdl, valset, seed)

            model = create_model('hrnet_32', heads =  {'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}, 
                                 head_conv = 256,num_gc_cls = None, 
                                 clsID4GC = None, gc_with_roi = False)
            
            if mdl == 'macaquepose':
                epoch *= 2

            #model = load_model(model, '../models/mcqcp/model_150.pth')
            model = load_model(model, '../exp/mot/'+mdl+'_seed'+seed+'/model_'+str(epoch)+'.pth')
            model = model.to(torch.device('cuda'))
            model.eval()

            path = "/path/to/root/"

            dataloader = datasets.LoadImagesAndBoxes(path, 
                                                     "../src/data/macaquepose.val",
                                                    valset = valset)

            mean_ap = MeanAveragePrecision()

            for i, (img_path, img, img0,labels) in enumerate(dataloader):
                if i == 500:
                    break


                im_blob = torch.from_numpy(img).cuda().unsqueeze(0)

                width = img0.shape[1]
                height = img0.shape[0]
                inp_height = im_blob.shape[2]
                inp_width = im_blob.shape[3]
                h_out = inp_height // 4
                w_out = inp_width // 4


                with torch.no_grad():
                    output = model(im_blob)[-1]
                    hm = output['hm'].sigmoid_()
                    wh = output['wh']
                    id_feature = output['id']
                    id_feature = F.normalize(id_feature, dim=1)
                    reg = output['reg']

                dets, inds, cls_inds_mask = mot_decode(heatmap=hm,
                                                       wh=wh,
                                                       reg=reg,
                                                       num_classes=1,
                                                       cat_spec_wh=False,
                                                       K=50)

                # translate and scale
                dets = map2orig(dets, h_out, w_out, height, width, 1)
                dets = dets[0]

                remain_inds = dets[:, 4] > 0.01
                dets = dets[remain_inds]
                #print("Total detections: ", len(dets), " - gt: ", len(labels))

                #ground truth
                boxes = torch.Tensor(labels[:,2:6] * np.array([width, height, width, height]))
                boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy")



                mean_ap.update(
                    [dict(boxes=torch.Tensor(dets[:,0:4]), scores=torch.Tensor(dets[:,4]), labels=torch.ones(len(dets[:,5])))],
                    [dict(boxes=boxes, labels=torch.ones(len(boxes)))],
                )

            scores = mean_ap.compute()
            print({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'], 'AP50': scores['map_50'], 'AP75': scores['map_75']})
            df = pd.concat([df, pd.DataFrame({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'].numpy(), 'AP50': scores['map_50'].numpy(), 'AP75': scores['map_75'].numpy()}, index = [0])])


In [ ]:
df['AP'] = [x.numpy() for x in df['AP']]
df['AP50'] = [x.numpy() for x in df['AP50']]
df['AP75'] = [x.numpy() for x in df['AP75']]

In [6]:
df.to_csv("../models200_detection.csv")